# Ugesco : enrich and desambiguate a sample of the Samnang-Hans' JSON

In [48]:
from ugesco import *

In [49]:
#test sur un Json contenant 20 éléments pris au hasard
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\jsondata_ugesco.json", "r", encoding="utf8") as file:
    json = json.load(file)
    
sample = random.sample(json, k=50)


In [50]:
data = pd.DataFrame([flatten(line) for line in sample])

In [51]:
data.head()

,beeldid,imageclassification_class1,imageclassification_class2,imageclassification_class3,imageclassification_class4,imageclassification_class5,imageclassification_prob1,imageclassification_prob2,imageclassification_prob3,imageclassification_prob4,...,spatial_key_1,spatial_key_2,spatial_value_0,spatial_value_1,spatial_value_2,temporal,temporal_key_0,temporal_key_1,temporal_value_0,temporal_value_1
0,00029072,plaza,slum,fountain,courtyard,street,0.325046,0.189850,0.090821,0.076835,...,LOC_ROADS,NaN,Bruxelles,Chaussée d' Alsemberg,NaN,NaN,TIMEX_DATE,TIMEX_TIME,3 septembre,5 heures du soir
1,00032588,street,plaza,crosswalk,downtown,alley,0.615152,0.163376,0.075181,0.014808,...,LOC_PLACE,NaN,Belge,Londres,NaN,NaN,TIMEX_DATE,NaN,21 juillet,NaN
2,00028740,street,railroad_track,loading_dock,alley,train_station/platform,0.273455,0.205406,0.184053,0.091288,...,LOC_PLACE,NaN,Palais,Anvers,NaN,NaN,NaN,NaN,NaN,NaN
3,00029118,bazaar/outdoor,plaza,stage/outdoor,amphitheater,beer_hall,0.243597,0.186574,0.165854,0.113516,...,LOC_STREET,NaN,Bruxelles,Bd de Waterloo,NaN,NaN,TIMEX_DATE,NaN,4 sept 44,NaN
4,00035650,plaza,apartment_building/outdoor,office_building,street,downtown,0.303153,0.162252,0.107686,0.101452,...,LOC_MONUMENT,NaN,Bruxelles,monument au pigeon-soldat de la grande guerre,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# exclude image classifications and temporal values for the moment
data = data[data.columns.drop(list(data.filter(regex='imageclassification|temporal')))]

In [53]:
# reshape the dataframe verticaly
data = data.set_index('beeldid')
data.columns = data.columns.str.split('_', expand=True)

In [54]:
data = data.stack().reset_index(level=1, drop=True).reset_index()

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [55]:
data.head()

beeldid                spatial                                     
                             value           key                  value
0  00029072              Bruxelles     LOC_PLACE              Bruxelles
1  00029072  Chaussée d' Alsemberg     LOC_ROADS  Chaussée d' Alsemberg
2  00032588                  Belge     LOC_PLACE                  Belge
3  00032588                Londres     LOC_PLACE                Londres
4  00028740                 Palais  LOC_BUILDING                 Palais

In [56]:
data.columns = ['_'.join(col) for col in data.columns]

In [57]:
data.rename(columns={'beeldid_': 'beeldid'}, inplace=True)

In [58]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value
0,00029072,Bruxelles,LOC_PLACE,Bruxelles
1,00029072,Chaussée d' Alsemberg,LOC_ROADS,Chaussée d' Alsemberg
2,00032588,Belge,LOC_PLACE,Belge
3,00032588,Londres,LOC_PLACE,Londres
4,00028740,Palais,LOC_BUILDING,Palais


In [59]:
#join/merge with phototheque_pallas to get the locations. 
#The csv is gzipped to get around the 100MB file weigth limitation of Github
phototeque = pd.read_csv("data/phototheque_pallas.csv.gz", compression="gzip", encoding="utf8", dtype={'beeldid': str, 'trfwnumm': str})

data = pd.merge(data, phototeque, how='left', on=['beeldid'], suffixes=['', '_x'])

In [60]:
# Pictures that contains more than a loc in the thesaurus descriptors

#data[data['loc_qid'].str.contains(",", na = False)]

In [61]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name
0,00029072,Bruxelles,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles
1,00029072,Chaussée d' Alsemberg,LOC_ROADS,Chaussée d' Alsemberg,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles
2,00032588,Belge,LOC_PLACE,Belge,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN
3,00032588,Londres,LOC_PLACE,Londres,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN
4,00028740,Palais,LOC_BUILDING,Palais,http://pallas.cegesoma.be/pls/opac/opac.search...,"(Foto Marcel Dewalhens) Antwerpen : ""Le Palais...","Libération d'Antwerpen par les alliés, 9/1944....",1736,NaN,Q31,BE,Belgique,Q12892,Anvers


In [62]:
# Merge with NER_classes (spatial_keys matched with wikidata)
ner_classes = pd.read_csv("data/ner_classes.csv")

data = pd.merge(data, ner_classes, how='left', left_on = ['spatial_key'], right_on = ['ner_class'])

In [63]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class,wiki_qid,wiki_class
0,00029072,Bruxelles,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,LOC_PLACE,Q56061,administrative territorial entity
1,00029072,Chaussée d' Alsemberg,LOC_ROADS,Chaussée d' Alsemberg,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,LOC_ROADS,Q83620,thoroughfare
2,00032588,Belge,LOC_PLACE,Belge,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN,LOC_PLACE,Q56061,administrative territorial entity
3,00032588,Londres,LOC_PLACE,Londres,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN,LOC_PLACE,Q56061,administrative territorial entity
4,00028740,Palais,LOC_BUILDING,Palais,http://pallas.cegesoma.be/pls/opac/opac.search...,"(Foto Marcel Dewalhens) Antwerpen : ""Le Palais...","Libération d'Antwerpen par les alliés, 9/1944....",1736,NaN,Q31,BE,Belgique,Q12892,Anvers,LOC_BUILDING,Q41176,building


In [64]:
# rename the new columns merged
data.rename(columns={'wiki_qid': 'ner_class_qid', 'wiki_class':'ner_class_name'}, inplace=True)

In [65]:
#remove duplicate columns
data = data.T.drop_duplicates().T

In [66]:
data.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00029072,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
1,00029072,Chaussée d' Alsemberg,LOC_ROADS,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
2,00032588,Belge,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00032588,Londres,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
4,00028740,Palais,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,"(Foto Marcel Dewalhens) Antwerpen : ""Le Palais...","Libération d'Antwerpen par les alliés, 9/1944....",1736,NaN,Q31,BE,Belgique,Q12892,Anvers,Q41176,building


In [67]:
# remove articles in spatial_value
data['spatial_value'] = data['spatial_value'].str.replace(pat=r"\b(le|la|l\s+'|l'|les)\b", repl="", n=1, case=False)

In [68]:
#data.to_csv(r"C:\Users\ettor\Desktop\data_ugesco.csv", header=True, index=False, encoding="utf-8")

In [69]:
data.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00029072,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
1,00029072,Chaussée d' Alsemberg,LOC_ROADS,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
2,00032588,Belge,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00032588,Londres,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
4,00028740,Palais,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,"(Foto Marcel Dewalhens) Antwerpen : ""Le Palais...","Libération d'Antwerpen par les alliés, 9/1944....",1736,NaN,Q31,BE,Belgique,Q12892,Anvers,Q41176,building


In [70]:
#normalize spatial_value
data['spatial_value'] = data['spatial_value'].apply(lambda x: Fingerprinter(x).get_fingerprint_nonsorted())

In [71]:
#transformer les loc_place pays en loc_country
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\countries.csv", 'r', encoding="utf8") as f:
    countries = set([Fingerprinter(country).get_fingerprint_nonsorted() for country in f])

# Grab DataFrame rows where column value is present in a list
data.loc[(data['spatial_value'].isin(countries)), ['spatial_key']] = "LOC_COUNTRY"

In [72]:
data

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00029072,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
1,00029072,chaussee ' alsemberg,LOC_ROADS,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
2,00032588,belge,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00032588,londres,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
4,00028740,palais,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,"(Foto Marcel Dewalhens) Antwerpen : ""Le Palais...","Libération d'Antwerpen par les alliés, 9/1944....",1736,NaN,Q31,BE,Belgique,Q12892,Anvers,Q41176,building
5,00028740,anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"(Foto Marcel Dewalhens) Antwerpen : ""Le Palais...","Libération d'Antwerpen par les alliés, 9/1944....",1736,NaN,Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity
6,00029118,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"4 sept 44 libération de Bruxelles : ""Les angla...","Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
7,00029118,bd de waterloo,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,"4 sept 44 libération de Bruxelles : ""Les angla...","Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
8,00035650,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Inauguration d'un monument au pigeo...,Monuments à Bruxelles : [...-1939]. - 16 photos,44817,monuments--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
9,00035650,monument au pigeon-soldat de grande guerre,LOC_MONUMENT,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Inauguration d'un monument au pigeo...,Monuments à Bruxelles : [...-1939]. - 16 photos,44817,monuments--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q4989906,monument


In [73]:
gb = data.groupby(['beeldid'])

In [74]:
gb.head() 

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00029072,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
1,00029072,chaussee ' alsemberg,LOC_ROADS,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
2,00032588,belge,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00032588,londres,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
4,00028740,palais,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,"(Foto Marcel Dewalhens) Antwerpen : ""Le Palais...","Libération d'Antwerpen par les alliés, 9/1944....",1736,NaN,Q31,BE,Belgique,Q12892,Anvers,Q41176,building
5,00028740,anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"(Foto Marcel Dewalhens) Antwerpen : ""Le Palais...","Libération d'Antwerpen par les alliés, 9/1944....",1736,NaN,Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity
6,00029118,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"4 sept 44 libération de Bruxelles : ""Les angla...","Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
7,00029118,bd de waterloo,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,"4 sept 44 libération de Bruxelles : ""Les angla...","Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
8,00035650,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Inauguration d'un monument au pigeo...,Monuments à Bruxelles : [...-1939]. - 16 photos,44817,monuments--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
9,00035650,monument au pigeon-soldat de grande guerre,LOC_MONUMENT,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Inauguration d'un monument au pigeo...,Monuments à Bruxelles : [...-1939]. - 16 photos,44817,monuments--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q4989906,monument


In [75]:
#Sélectionner les records qui contiennent moins de 3 loc_place (attention : brussels et bruxelles = 1 en fait. D'où, deux semble un bon compromis)
df_test = gb['spatial_key'].apply(lambda x: (x=='LOC_PLACE').sum()).reset_index(name='count_places')

In [76]:
df_test

,beeldid,count_places
0,00000063,1
1,00004364,1
2,00004782,1
3,00005594,0
4,00009533,0
5,00009971,1
6,00010712,1
7,00010969,1
8,00010991,1
9,00012338,1


In [77]:
filtre = list(df_test[df_test['count_places'] < 3]['beeldid'])  
gb_filtered = data[data['beeldid'].isin(filtre)].groupby('beeldid')

In [78]:
gb_filtered.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00029072,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
1,00029072,chaussee ' alsemberg,LOC_ROADS,http://pallas.cegesoma.be/pls/opac/opac.search...,Libération Bruxelles 3 septembre 5 heures du s...,"Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
2,00032588,belge,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
3,00032588,londres,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Fête Nationale Belge, Londres, 21 juillet 1934...","Hubert Pierlot, homme politique belge, premier...",627,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
4,00028740,palais,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,"(Foto Marcel Dewalhens) Antwerpen : ""Le Palais...","Libération d'Antwerpen par les alliés, 9/1944....",1736,NaN,Q31,BE,Belgique,Q12892,Anvers,Q41176,building
5,00028740,anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"(Foto Marcel Dewalhens) Antwerpen : ""Le Palais...","Libération d'Antwerpen par les alliés, 9/1944....",1736,NaN,Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity
6,00029118,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"4 sept 44 libération de Bruxelles : ""Les angla...","Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
7,00029118,bd de waterloo,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,"4 sept 44 libération de Bruxelles : ""Les angla...","Libération de Bruxelles par les allies., 9/194...",1625,bruxelles (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q239,Bruxelles,Q83620,thoroughfare
8,00035650,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Inauguration d'un monument au pigeo...,Monuments à Bruxelles : [...-1939]. - 16 photos,44817,monuments--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q56061,administrative territorial entity
9,00035650,monument au pigeon-soldat de grande guerre,LOC_MONUMENT,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Inauguration d'un monument au pigeo...,Monuments à Bruxelles : [...-1939]. - 16 photos,44817,monuments--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q4989906,monument


In [79]:
#et maintenant appliquer la concatenation uniquement si le record ne contient pas plus de deux loc_place
gb = pd.DataFrame(gb_filtered['spatial_value'].apply(lambda x: "%s" % ' '.join(x))).reset_index()

In [80]:
gb.head()

,beeldid,spatial_value
0,00000063,cathedrale tournai
1,00004364,bruxelles rue des colonies
2,00004782,anvers allemagne
3,00005594,bourse
4,00009533,beffroi ' ypres


In [81]:
#comment concaténer loc_name maintenant, et accéder à country_name ?
#en attendant de trouver une solution élégante, récréer un dataset contenant ces infos

gb = gb.merge(data[["beeldid", "loc_name", "country_name"]])

In [82]:
gb.head()

,beeldid,spatial_value,loc_name,country_name
0,00000063,cathedrale tournai,Tournai,Belgique
1,00000063,cathedrale tournai,Tournai,Belgique
2,00004364,bruxelles rue des colonies,NaN,Belgique
3,00004364,bruxelles rue des colonies,NaN,Belgique
4,00004782,anvers allemagne,NaN,Belgique


In [83]:
#on splitte les noms de pays multiples (n'existent plus)
#gb = tidy_split(gb.merge(data[["beeldid", "country_name", "loc_name"]]), column="country_name", sep=",", keep=False)

In [84]:
#on enlève les doublons
gb.drop_duplicates(subset=None, keep='first', inplace=True)

In [85]:
gb

,beeldid,spatial_value,loc_name,country_name
0,00000063,cathedrale tournai,Tournai,Belgique
2,00004364,bruxelles rue des colonies,NaN,Belgique
4,00004782,anvers allemagne,NaN,Belgique
6,00005594,bourse,Bruxelles,Belgique
7,00009533,beffroi ' ypres,Ypres,Belgique
8,00009971,ostende,NaN,Belgique
9,00010712,malines,Elewijt,Belgique
10,00010969,bruxelles ' eglise st-jacques,Bruxelles,Belgique
12,00010991,bruxelles grand'place,Bruxelles,Belgique
14,00012338,bruxelles grand'place,Bruxelles,Belgique


In [86]:
#on normalise les lieux et pays
gb['loc_name'] = gb['loc_name'].apply(lambda x: Fingerprinter(str(x)).get_fingerprint_nonsorted()).str.replace('nan',  '')
gb['country_name'] = gb['country_name'].apply(lambda x: Fingerprinter(str(x)).get_fingerprint_nonsorted()).str.replace('nan',  '')

In [87]:
# on concatène le tout en supprimant les doublons

gb['loc_concat'] = gb['spatial_value'].fillna('').map(
    str) + ' ' + gb['loc_name'].fillna('').map(str) + ' ' + gb['country_name'].fillna('').map(str)

gb['loc_concat'] = gb['loc_concat'].apply(
    lambda x: ' '.join(pd.unique(x.split())))

In [88]:
gb

,beeldid,spatial_value,loc_name,country_name,loc_concat
0,00000063,cathedrale tournai,tournai,belgique,cathedrale tournai belgique
2,00004364,bruxelles rue des colonies,,belgique,bruxelles rue des colonies belgique
4,00004782,anvers allemagne,,belgique,anvers allemagne belgique
6,00005594,bourse,bruxelles,belgique,bourse bruxelles belgique
7,00009533,beffroi ' ypres,ypres,belgique,beffroi ' ypres belgique
8,00009971,ostende,,belgique,ostende belgique
9,00010712,malines,elewijt,belgique,malines elewijt belgique
10,00010969,bruxelles ' eglise st-jacques,bruxelles,belgique,bruxelles ' eglise st-jacques belgique
12,00010991,bruxelles grand'place,bruxelles,belgique,bruxelles grand'place belgique
14,00012338,bruxelles grand'place,bruxelles,belgique,bruxelles grand'place belgique


In [89]:
#appliquer get_nominatim. ATTENTION : ajouter le pays pour plus de précision

gb['nominatim'] = gb['loc_concat'].apply(get_nominatim)

D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=3792, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:886d:dfde:a57f:2129', 52996, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=3792, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:886d:dfde:a57f:2129', 52998, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=3796, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('2a02:1811:3700:1500:886d:dfde:a57f:2129', 53000, 0, 0), raddr=('2001:630:12:500:ec4:7aff:fe66:96d2', 443, 0, 0)>
  self._sock = None
D:\Anaconda3\lib\socket.py:657: ResourceWarning: unclosed <socket.socket fd=3808, family=AddressFamily.AF_INET6, type=Socket

In [90]:
gb.head(10)

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim
0,00000063,cathedrale tournai,tournai,belgique,cathedrale tournai belgique,"[{'place_id': '85444409', 'licence': 'Data © O..."
2,00004364,bruxelles rue des colonies,,belgique,bruxelles rue des colonies belgique,[]
4,00004782,anvers allemagne,,belgique,anvers allemagne belgique,[]
6,00005594,bourse,bruxelles,belgique,bourse bruxelles belgique,"[{'place_id': '68588319', 'licence': 'Data © O..."
7,00009533,beffroi ' ypres,ypres,belgique,beffroi ' ypres belgique,[]
8,00009971,ostende,,belgique,ostende belgique,"[{'place_id': '178191862', 'licence': 'Data © ..."
9,00010712,malines,elewijt,belgique,malines elewijt belgique,[]
10,00010969,bruxelles ' eglise st-jacques,bruxelles,belgique,bruxelles ' eglise st-jacques belgique,[]
12,00010991,bruxelles grand'place,bruxelles,belgique,bruxelles grand'place belgique,"[{'place_id': '68150353', 'licence': 'Data © O..."
14,00012338,bruxelles grand'place,bruxelles,belgique,bruxelles grand'place belgique,"[{'place_id': '68150353', 'licence': 'Data © O..."


In [91]:
#Comment attribuer un score de fiabilité aux résultats et choisir le meilleur s'il y en a un ?

def get_element(array, *args):
    """parse the first nominatim json."""
    for el in array:
        return dig(el, *args)
    
gb['nominatim_main_loc'] = np.vectorize(get_element)(gb['nominatim'], "display_name")
gb['nominatim_type_loc'] = np.vectorize(get_element)(gb['nominatim'], "type")
gb['nominatim_city_loc'] = np.vectorize(get_element)(gb['nominatim'], "address", "city")
gb['nominatim_country_loc'] = np.vectorize(get_element)(gb['nominatim'], "address", "country") 

In [92]:
gb 

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,nominatim_main_loc,nominatim_type_loc,nominatim_city_loc,nominatim_country_loc
0,00000063,cathedrale tournai,tournai,belgique,cathedrale tournai belgique,"[{'place_id': '85444409', 'licence': 'Data © O...","Cathédrale Notre-Dame de Tournai, Place Paul-E...",place_of_worship,None,Belgique
2,00004364,bruxelles rue des colonies,,belgique,bruxelles rue des colonies belgique,[],None,None,None,None
4,00004782,anvers allemagne,,belgique,anvers allemagne belgique,[],None,None,None,None
6,00005594,bourse,bruxelles,belgique,bourse bruxelles belgique,"[{'place_id': '68588319', 'licence': 'Data © O...","Bourse, Rue de la Bourse, Quartier du Centre, ...",attraction,Ville de Bruxelles,Belgique
7,00009533,beffroi ' ypres,ypres,belgique,beffroi ' ypres belgique,[],None,None,None,None
8,00009971,ostende,,belgique,ostende belgique,"[{'place_id': '178191862', 'licence': 'Data © ...","Ostende, Flandre-Occidentale, Flandre, 8400, B...",administrative,Ostende,Belgique
9,00010712,malines,elewijt,belgique,malines elewijt belgique,[],None,None,None,None
10,00010969,bruxelles ' eglise st-jacques,bruxelles,belgique,bruxelles ' eglise st-jacques belgique,[],None,None,None,None
12,00010991,bruxelles grand'place,bruxelles,belgique,bruxelles grand'place belgique,"[{'place_id': '68150353', 'licence': 'Data © O...","Hôtel de ville de Bruxelles, 8, Grand-Place, Q...",attraction,Ville de Bruxelles,Belgique
14,00012338,bruxelles grand'place,bruxelles,belgique,bruxelles grand'place belgique,"[{'place_id': '68150353', 'licence': 'Data © O...","Hôtel de ville de Bruxelles, 8, Grand-Place, Q...",attraction,Ville de Bruxelles,Belgique


In [93]:
gb['wikipedia_improved'] = gb[gb.main_loc == None]['loc_concat'].apply(lambda x: gsrsearch(x))

AttributeError: 'DataFrame' object has no attribute 'main_loc'

In [ ]:
gb